In [ ]:
import arviz as az
import matplotlib.pyplot as plt
import numpy as np
import pymc as pm
import xarray as xr

from scipy.special import expit as logistic


print(f"Running on PyMC v{pm.__version__}")

In [ ]:
az.style.use("arviz-darkgrid")

RANDOM_SEED = 58
rng = np.random.default_rng(RANDOM_SEED)


def standardize(series):
    """Standardize a pandas series"""
    return (series - series.mean()) / series.std()

In [ ]:
N = 400
true_intercept = 0.2
true_slope = 1.0
predictors = rng.normal(size = N)
true_p = logistic(true_intercept + true_slope * predictors)

outcomes = rng.binomial(1, true_p)
outcomes[:10]

In [ ]:
with pm.Model() as model_2:
    betas = pm.Normal("betas", mu=0.0, sigma=np.array([0.5, 1.0]), shape=2)

    # set predictors as shared variable to change them for PPCs:
    pred = pm.MutableData("pred", predictors, dims="obs_id")
    p = pm.Deterministic("p", pm.math.invlogit(betas[0] + betas[1] * pred), dims="obs_id")

    outcome = pm.Bernoulli("outcome", p=p, observed=outcomes, dims="obs_id")

    idata_2 = pm.sample(1000, tune=2000, return_inferencedata=True, random_seed=rng)
az.summary(idata_2, var_names=["betas"], round_to=2)

In [ ]:
predictors_out_of_sample = rng.normal(size=50)
outcomes_out_of_sample = rng.binomial(
    1, logistic(true_intercept + true_slope * predictors_out_of_sample)
)

with model_2:
    # update values of predictors:
    pm.set_data({"pred": predictors_out_of_sample})
    # use the updated values and predict outcomes and probabilities:
    idata_2 = pm.sample_posterior_predictive(
        idata_2,
        var_names=["p"],
        return_inferencedata=True,
        predictions=True,
        extend_inferencedata=True,
        random_seed=rng,
    )

In [ ]:
_, ax = plt.subplots(figsize=(12, 6))

preds_out_of_sample = idata_2.predictions_constant_data.sortby("pred")["pred"]
model_preds = idata_2.predictions.sortby(preds_out_of_sample)

# uncertainty about the estimates:
ax.vlines(
    preds_out_of_sample,
    *az.hdi(model_preds)["p"].transpose("hdi", ...),
    alpha=0.8,
)
# expected probability of success:
ax.plot(
    preds_out_of_sample,
    model_preds["p"].mean(("chain", "draw")),
    "o",
    ms=5,
    color="C1",
    alpha=0.8,
    label="Expected prob.",
)

# actual outcomes:
ax.scatter(
    x=predictors_out_of_sample,
    y=outcomes_out_of_sample,
    marker="x",
    color="k",
    alpha=0.8,
    label="Observed outcomes",
)
# true probabilities:
x = np.linspace(predictors_out_of_sample.min() - 0.1, predictors_out_of_sample.max() + 0.1)
ax.plot(
    x,
    logistic(true_intercept + true_slope * x),
    lw=2,
    ls="--",
    color="#565C6C",
    alpha=0.8,
    label="True prob.",
)

ax.set_xlabel("Predictor")
ax.set_ylabel("Prob. of success")
ax.set_title("Out-of-sample Predictions")
ax.legend(fontsize=10, frameon=True, framealpha=0.5);

In [ ]:
# Above: https://www.pymc.io/projects/docs/en/stable/learn/core_notebooks/posterior_predictive.html

In [ ]:
y_pred = post_checks["yl"].mean(0)
jitter = np.random.normal(0, 0.03, len(y_s))

plt.figure(figsize=(12, 5))
plt.scatter(y_s + jitter, y_pred, alpha=0.4)
plt.xticks(range(3), iris.species.unique())
plt.xlabel("Observed category")
plt.yticks(range(3), iris.species.unique())
plt.ylabel("Predicted category")
plt.title("In-sample posterior predictive check");

We see that the model does a very good job at predicting setosa flowers, but seems to have trouble differentiating versicolor from virginica - it regularly mixes up one for the other. This suggests that these two species are close regarding the features we included into the model. If we had a finer view of what tells them apart, we could get this information into the model and improve our predictions - and understanding. How do we do that? Well, we're no botanists, so that's where domain knowledge comes into play.

More generally, this notebook focused on how to inspect the gut of your model for not-so-simple models, like mutinomial regressions. But we did not think causally here to determine which predictors to regress on. That's ok for a notebook that aims to give a technical view, but if you want to go beyond mere predictions - e.g understanding the process and intervening in the real world based on your model's insights - then you'll need a causal graph before doing any modeling.

You're still there? Well, thanks for reading! As we are (still) mere mortals, we may have forgotten about important aspects or made mistakes - so please, send your pull requests our way on this repo. And above all, live long & PyMCheers!

In [ ]:
%watermark -a AlexAndorra -n -u -v -iv -w